In [ ]:
!pip install pysal

In [ ]:
import numpy as np
import pandas as pd
import libpysal.weights as lpw

# SAR
$$ 
y = \rho W y + v  \\
y = (I-\rho W)^{-1}v   
$$








## 1. Weight matrix


In [ ]:
# Create a set of coordinates for the spatial units

# Define the range of coordinate values
xmin, ymin = 0, 0
xmax, ymax = 10, 10

# Set the number of spatial units
n_units = 3

# Create a set of random coordinates for n_units
x_coords = np.random.uniform(xmin, xmax, size=n_units)
y_coords = np.random.uniform(ymin, ymax, size=n_units)
coords = np.column_stack((x_coords, y_coords))

# Print the resulting coordinates
print(coords)

[[9.69544986 4.79915455]
 [4.11107595 1.94288259]
 [3.47878334 3.92121443]]


In [ ]:
# Calculate distances between coordinates

# Initialize an empty distances matrix
n = len(coords)
distances = np.zeros((n, n))

# Compute the euclidean distance between pair of coords
for i in range(n):
    for j in range(i+1, n):
        distances[i,j] = distances[j,i] = np.linalg.norm(coords[i] - coords[j])       

print(distances)

[[0.         6.27244142 6.2783534 ]
 [6.27244142 0.         2.07691858]
 [6.2783534  2.07691858 0.        ]]


In [ ]:
# Create weight matrix using PySAL

# Create a distance-based weight matrix using inverse distance
w = lpw.DistanceBand.from_array(coords, threshold=7, binary=True)

# Convert the weight matrix to row-stochastic form
w.transform = 'R'

# Print the weight matrix
print(w.full())

(array([[0. , 0.5, 0.5],
       [0.5, 0. , 0.5],
       [0.5, 0.5, 0. ]]), [0, 1, 2])


## 2. Independent variable $v$
* $3 \times 1$ vector
* Randomly generated from standard normal distribution

In [ ]:
v = np.random.randn(3, 1)
print(x)

[[-1.38707183]
 [ 1.62772069]
 [-0.18544584]]


## 3. Generated variable $\bar{y}$
* Generate by $\bar{y} = (I-\rho W)^{-1}v$
* Assume that $ \rho = 0.1$

In [ ]:
W = w.full()[0]

array([[0. , 0.5, 0.5],
       [0.5, 0. , 0.5],
       [0.5, 0.5, 0. ]])

In [ ]:
# Weight matrix
W = w.full()[0]

# Compute the matrix product
I = np.identity(W.shape[0])
rho = 0.1
A = np.linalg.inv(I - rho * W)

y_bar = np.dot(A, v)

print(y_bar.T)

[[0.98124248 0.58660642 0.18895996]]


## 4. Simulation

In [ ]:
# Set up parameters
num_simulations = 1000
num_points = 3 # len(coords)

W = w.full()[0]
I = np.identity(W.shape[0])
rho = 0.1
A = np.linalg.inv(I - rho * W)

results = []

# Run simulations
for i in range(num_simulations):
    # Generate random v from standard normal distribution
    v = np.random.randn(3, 1) 
    # Compute the matrix product
    y = np.dot(A, v)
    results.append(y)

# Make a column names list; y1, y2, ..
col_names = []

for i in range(num_points):
    col_names.append('y{}'.format(i+1))

# Convert results to dataframe
simul_df = pd.DataFrame(np.reshape(results, (num_simulations,num_points)), columns=col_names)

print(simul_df)

           y1        y2        y3
0   -0.248182 -0.814621 -0.872107
1   -0.986431 -1.583850 -0.664038
2    1.302061  1.505517  0.145194
3   -0.854931 -1.235879  2.602423
4    0.245954  0.662011  0.165104
..        ...       ...       ...
995  0.395872  0.038296 -0.578386
996  2.134512  0.754965  1.543790
997 -0.097262 -0.678311  0.188035
998 -0.902792 -0.787287 -0.517793
999  1.390296  0.276648 -0.428744

[1000 rows x 3 columns]


# Conditional Covariance
1. Covariance of generated $\bar{y}$
2. Calculated conditional covariance by using the model $$Cov(y_{i}, y_{j}|X) = \sum_{j}A_{ij}A_{kj}$$

## 1. Covariance of generated $\bar{y}$

In [ ]:
cov_simul = simul_df.cov()
cov_simul

,y1,y2,y3
y1,0.998956,0.093652,0.179834
y2,0.093652,1.119664,0.139698
y3,0.179834,0.139698,0.971981


## 2. Calculated conditional covariance by using the model

In [ ]:
# Initialize an empty covariance matrix
n = len(A)
cov_model = np.zeros((n, n))


for i in range(n):
  for k in range(i, n):
    for j in range(n):
      cov_model[i,k] = cov_model[k,i] = np.sum(A[i,j]*A[k,j]) # cf. sigma = 1

# Convert results to dataframe
cov_model = pd.DataFrame(cov_model, columns=col_names, index=col_names)

cov_model

,y1,y2,y3
y1,0.002799,0.002799,0.05319
y2,0.002799,0.002799,0.05319
y3,0.053190,0.053190,1.01061


In [ ]:
# Calculate the Frobenius norm of the difference between the matrices
diff_norm = np.linalg.norm(cov_model - cov_simul, ord='fro')
diff_norm

1.5181436713221714